모듈 설치

In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0sqa0afi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0sqa0afi
  Resolved https://github.com/huggingface/transformers to commit f4d57f2f0cdff0f63ee74a1f16f442dfaf525231
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 215.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 243.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.56.0.dev0-py3-none-any.whl size=12261147 sha256=31ff1ddb0966328258a8ef84a3d817caabc43331405657c8dd6aa580d602d0b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-0qilsrpw/

In [2]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 249.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


import

In [3]:
!pip install datasets
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 636.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 595.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 407.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 262.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 624.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import io
import json
from PIL import Image
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
#from IPython.display import display


In [5]:
from transformers import AutoProcessor, AutoModelForVision2Seq
from qwen_vl_utils import process_vision_info
import torch

검증용 데이터 로드

In [6]:
dataset = load_dataset("ashraq/fashion-product-images-small", split="train")


README.md:   0%|          | 0.00/867 [00:00<?, ?B/s]

(…)-00000-of-00002-6cff4c59f91661c3.parquet:   0%|          | 0.00/136M [00:00<?, ?B/s]

(…)-00001-of-00002-bb459e5ac5f01e71.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44072 [00:00<?, ? examples/s]

검증용 데이터에서 Label정보를 추출하여 별도의 데이터로 추가

In [7]:
def combine_cols_to_label(example):
    # 실제 컬럼명에 맞게 수정
    label_dict = {
        "gender": example["gender"],
        "masterCategory": example["masterCategory"],
        "subCategory": example["subCategory"],
        "season": example["season"],
        "usage": example["usage"],
        "baseColour": example["baseColour"],
        "articleType": example["articleType"],
    }
    example["label"] = json.dumps(label_dict, ensure_ascii=False)
    return example

In [8]:
dataset_add_label = dataset.map(combine_cols_to_label)
dataset_add_label = dataset_add_label.shuffle(seed=4242)

Map:   0%|          | 0/44072 [00:00<?, ? examples/s]

레이블 정보가 추가된 검증용 데이터에 대해서 system과 user prompt template를 추가하여 json 형식으로 변환

In [9]:
# 시스템(assistant)에게 주어진 역할
system_message = "당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다."

# 실제로 사용자 입력 -> 모델이 답해야 하는 프롬프트
prompt = """입력 정보:
- name: {name}
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}}

# 예시
{{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
"""

In [10]:
def format_data(sample):
   # Image.Image를 PngImageFile로 변환
   buffer = io.BytesIO()
   sample["image"].save(buffer, format='PNG')
   buffer.seek(0)
   image = Image.open(buffer)

   return {
       "messages": [
           {
               "role": "system",
               "content": [
                   {
                       "type": "text",
                       "text": system_message
                   }
               ],
           },
           {
               "role": "user",
               "content": [
                   {
                       "type": "text",
                       "text": prompt.format(name=sample["productDisplayName"]),
                   },
                   {
                       "type": "image",
                       "image": image,
                   }
               ],
           },
           {
               "role": "assistant",
               "content": [
                   {
                       "type": "text",
                       "text": sample["label"],
                   }
               ],
           },
       ],
   }

In [11]:
formatted_dataset = [format_data(row) for row in tqdm(dataset_add_label)]

100%|██████████| 44072/44072 [00:38<00:00, 1155.26it/s]


모델 로드

In [12]:
model_id = "mypsyche98/Qwen2-VL-7B-Instruct_LoRA_custom_20250805_merged"

In [13]:

model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2202: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

model-00004-of-00018.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

...

model-00016-of-00018.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

model-00017-of-00018.safetensors:   0%|          | 0.00/2.18G [00:00<?, ?B/s]

model-00018-of-00018.safetensors:   0%|          | 0.00/815M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/18 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

In [14]:
processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

preprocessor_config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

검증 테스트용 샘플링 데이터

In [15]:
sample_data = formatted_dataset[100]['messages']

In [16]:
sample_prompt = processor.apply_chat_template(
    sample_data,
    tokenize=False,
    add_generation_prompt=False,
)

프롬프트 템플릿이 적용된 데이터에서 질의와 레이블을 분리 => 모델에서 추론된 데이터와 레이블을 비교하기 위해서

In [17]:
def split_input_and_label(prompt):
    input = prompt.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = prompt.split('<|im_start|>assistant')[1]
    return input, label

In [18]:
sample_prompt_input, sample_prompt_label = split_input_and_label(sample_prompt)

샘플 데이터에서 image와 video 정보 분리

In [19]:
sample_data_image_inputs, sample_data_video_inputs = process_vision_info(sample_data)

VLM 질의용 객체 생성

In [20]:
inputs = processor(
    text=[sample_prompt_input],
    images=sample_data_image_inputs,
    videos=sample_data_video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

inference

In [21]:

# Inference: Generation of the output
generated_ids = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
)

generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print("추론결과=",output_text)


추론결과= ['\n{"gender": "Men", "masterCategory": "Apparel", "subCategory": "Innerwear", "season": "Summer", "usage": "Casual", "baseColour": "White", "articleType": "Innerwear Vests"}']


In [22]:
print("sample_prompt_label=",sample_prompt_label)

sample_prompt_label= 
{"gender": "Men", "masterCategory": "Apparel", "subCategory": "Innerwear", "season": "Summer", "usage": "Casual", "baseColour": "White", "articleType": "Innerwear Vests"}<|im_end|>



In [35]:
# 랜덤 50개 테스트 데이터 추출
import random

# 배열에서 랜덤하게 50개의 요소를 뽑아내는 코드
test_samples = random.sample(formatted_dataset, 50)

# 배치 인퍼런스를 위한 입력 데이터 준비
batch_prompts = []
batch_image_inputs = []
batch_video_inputs = []
batch_labels = []

for sample in test_samples:
    # 각 샘플의 메시지 처리
    messages = sample["messages"]

    # 프롬프트 생성
    prompt = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )

    # 입력과 레이블 분리
    input_part = prompt.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label_part = prompt.split('<|im_start|>assistant')[1]

    # 이미지/비디오 데이터 처리
    image_inputs, video_inputs = process_vision_info(messages)

    # 모든 프롬프트와 기타 데이터를 리스트에 추가
    batch_prompts.append(input_part)
    batch_image_inputs.append(image_inputs)
    batch_labels.append(label_part)

# 루프가 끝난 후, 모든 데이터를 한 번에 processor에 전달합니다.
# 이렇게 하면 길이가 다른 프롬프트들을 자동으로 가장 긴 길이에 맞춰 패딩합니다.
batch_inputs = processor(
    text=batch_prompts,
    images=batch_image_inputs,
    videos=None,
    padding=True,
    return_tensors="pt",
).to("cuda")


In [36]:

# Inference: Generation of the output
generated_ids = model.generate(
    **batch_inputs,
    max_new_tokens=128,
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
)

generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(batch_inputs.input_ids, generated_ids)]
output_texts = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)

print("output_texts=",output_texts)


output_texts= ['\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n\n출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:\n{"gender": "Men", "masterCategory": "Footwear", "subCategory": "Shoes", "season": "Summer", "usage": "Sports", "baseColour": "White", "articleType": "Sports Shoes"}\n\n위 JSON 예시 형태를 지키지 않으면 아래와 같이 오류 메시지가 출력됩니다:\n{"error": "입력 정보가 정확하지', '\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n\n출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:\n{"gender": "Men", "masterCategory": "Footwear", "subCategory": "Shoes", "season": "Summer", "usage": "Sports", "baseColour": "White", "articleType": "Sports Shoes"}\n\n위 JSON 예시 형태를 지키지 않으면 아래와 같이 오류 메시지가 출력됩니다:\n{"error": "입력 정보가 정확하지', '\n- 7개 항목 이외의 정보(텍스트 등)가 포함되어 있습니다.', '\n{"gender": "Men', '\n{"gender": "Men", "masterCategory": "Apparel", "subCategory": "Bottomwear", "season": "Fall", "usage": "Casual", "baseColour": "Blue", "articleType": "Jeans"}', '\n- JSON 형태 이외의 형태(텍스트, 문장 등)은 절대 포함하지 마세요.\n\n출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:\n{"gender": "Men", "masterCategory": "App

In [37]:

# 결과 출력
print("\n=== 배치 인퍼런스 결과 ===")
for i, output in enumerate(output_texts):
    print(f"\n--- 샘플 {i+1} ---")
    print(f"모델의 예측: {output.strip()}")
    print(f"원래 레이블: {batch_labels[i].strip()}")
    print("-" * 50)



=== 배치 인퍼런스 결과 ===

--- 샘플 1 ---
모델의 예측: - 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{"gender": "Men", "masterCategory": "Footwear", "subCategory": "Shoes", "season": "Summer", "usage": "Sports", "baseColour": "White", "articleType": "Sports Shoes"}

위 JSON 예시 형태를 지키지 않으면 아래와 같이 오류 메시지가 출력됩니다:
{"error": "입력 정보가 정확하지
원래 레이블: {"gender": "Men", "masterCategory": "Footwear", "subCategory": "Shoes", "season": "Fall", "usage": "Sports", "baseColour": "White", "articleType": "Sports Shoes"}<|im_end|>
--------------------------------------------------

--- 샘플 2 ---
모델의 예측: - 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{"gender": "Men", "masterCategory": "Footwear", "subCategory": "Shoes", "season": "Summer", "usage": "Sports", "baseColour": "White", "articleType": "Sports Shoes"}

위 JSON 예시 형태를 지키지 않으면 아래와 같이 오류 메시지가 출력됩니다:
{"error": "입력 정보가 정확하지
원래 레이블: {"gender": "Men", "masterCategory": "Footwear", "subCategory": "Shoes", "season"

In [41]:
import json
from sklearn.metrics import f1_score
from collections import defaultdict

def calculate_f1_scores(outputs, batch_labels):
    """각 key별로 F1 score를 계산하는 함수"""

    # 각 키별로 예측값과 실제값을 저장할 딕셔너리
    predictions_by_key = defaultdict(list)
    labels_by_key = defaultdict(list)

    for i, output in enumerate(outputs):
        # 모델 예측 결과 파싱
        try:
            # `pred_text`가 이미 문자열이므로 바로 사용합니다.
            pred_text = output.strip()
            
            # JSON만 추출하기
            # 예측 결과에 불필요한 텍스트가 포함된 경우 JSON 부분만 찾아냅니다.
            start_index = pred_text.find('{')
            end_index = pred_text.rfind('}')
            
            if start_index != -1 and end_index != -1:
                json_string = pred_text[start_index : end_index + 1]
                pred_json = json.loads(json_string)
            else:
                print(f"샘플 {i+1}: 예측 결과에 JSON 형태가 없음")
                continue
        except json.JSONDecodeError:
            print(f"샘플 {i+1}: 예측 결과 JSON 파싱 실패")
            continue

        # 실제 레이블 파싱
        try:
            label_text = batch_labels[i].strip()
            # <|im_end|> 제거
            if label_text.endswith('<|im_end|>'):
                label_text = label_text[:-10]
            label_json = json.loads(label_text)
        except:
            print(f"샘플 {i+1}: 레이블 JSON 파싱 실패")
            continue

        # 각 키별로 예측값과 실제값 저장
        for key in label_json.keys():
            if key in pred_json:
                predictions_by_key[key].append(pred_json[key])
                labels_by_key[key].append(label_json[key])
            else:
                print(f"샘플 {i+1}: 키 '{key}' 예측 결과에 없음")

    # 각 키별로 F1 score 계산
    f1_scores = {}
    for key in labels_by_key.keys():
        if len(labels_by_key[key]) > 0:
            # 고유한 라벨들 확인
            unique_labels = list(set(labels_by_key[key] + predictions_by_key[key]))

            if len(unique_labels) == 1:
                # 모든 값이 동일한 경우
                f1_scores[key] = 1.0
            else:
                # F1 score 계산 (macro average)
                f1_scores[key] = f1_score(
                    labels_by_key[key],
                    predictions_by_key[key],
                    labels=unique_labels,
                    average='macro',
                    zero_division=0
                )

    return f1_scores, predictions_by_key, labels_by_key

# F1 score 계산 실행
f1_results, preds_by_key, labels_by_key = calculate_f1_scores(output_texts, batch_labels)

# 결과 출력
print("\n=== F1 Score 결과 ===")
print(f"{'Key':<15} {'F1 Score':<10} {'샘플 수':<8}")
print("-" * 35)

for key, score in f1_results.items():
    sample_count = len(labels_by_key[key])
    print(f"{key:<15} {score:.4f}     {sample_count}")

# 전체 평균 F1 score
if f1_results:
    avg_f1 = sum(f1_results.values()) / len(f1_results)
    print("-" * 35)
    print(f"{'평균 F1':<15} {avg_f1:.4f}")

# 각 키별 상세 결과 (옵션)
print("\n=== 상세 결과 ===")
for key in f1_results.keys():
    correct = sum(1 for p, l in zip(preds_by_key[key], labels_by_key[key]) if p == l)
    total = len(labels_by_key[key])
    accuracy = correct / total if total > 0 else 0
    print(f"{key}: 정확도 {accuracy:.4f} ({correct}/{total})")

샘플 3: 예측 결과에 JSON 형태가 없음
샘플 4: 예측 결과에 JSON 형태가 없음
샘플 11: 예측 결과에 JSON 형태가 없음
샘플 14: 예측 결과에 JSON 형태가 없음
샘플 15: 예측 결과 JSON 파싱 실패
샘플 17: 예측 결과에 JSON 형태가 없음
샘플 18: 예측 결과에 JSON 형태가 없음
샘플 21: 예측 결과에 JSON 형태가 없음
샘플 24: 예측 결과에 JSON 형태가 없음
샘플 25: 예측 결과에 JSON 형태가 없음
샘플 27: 예측 결과에 JSON 형태가 없음
샘플 28: 예측 결과에 JSON 형태가 없음
샘플 35: 예측 결과 JSON 파싱 실패
샘플 37: 예측 결과에 JSON 형태가 없음
샘플 39: 예측 결과 JSON 파싱 실패
샘플 42: 예측 결과에 JSON 형태가 없음
샘플 46: 예측 결과에 JSON 형태가 없음

=== F1 Score 결과 ===
Key             F1 Score   샘플 수    
-----------------------------------
gender          1.0000     33
masterCategory  1.0000     33
subCategory     1.0000     33
season          0.6070     33
usage           0.7316     33
baseColour      0.8320     33
articleType     0.8889     33
-----------------------------------
평균 F1           0.8656

=== 상세 결과 ===
gender: 정확도 1.0000 (33/33)
masterCategory: 정확도 1.0000 (33/33)
subCategory: 정확도 1.0000 (33/33)
season: 정확도 0.6667 (22/33)
usage: 정확도 0.8485 (28/33)
baseColour: 정확도 0.8788 (29/33)
articleType

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
/usr/local/lib/python3.11/dist-packages/sklearn/utils/multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/multiclass.py:79: UserWarning: The number of unique classes is greater t